In [1]:
pip install torch torch-geometric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

# Step 1: Define the graph structure
# Node features
x = torch.tensor([[1, 0],  # Node 1 features
                  [0, 1],  # Node 2 features
                  [1, 1]], # Node 3 features
                  dtype=torch.float)

# Edge indices (from-to relationships)
edge_index = torch.tensor([[0, 1, 1, 2],  # Edge from node 1 to 2, node 2 to 3
                           [1, 0, 2, 1]], dtype=torch.long)

# Step 2: Create the graph data
data = Data(x=x, edge_index=edge_index)

# Step 3: Define a simple GCN model with one layer
class GCN(nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        # 2 input features per node, 2 output features per node
        self.conv1 = GCNConv(2, 2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        # Apply GCN layer
        x = self.conv1(x, edge_index)
        x = F.relu(x)  # Apply ReLU activation
        return x

# Step 4: Create the model and perform a forward pass
model = GCN()
output = model(data)

# Step 5: Display the results
print(f"Node features after GCN layer:\n{output}")


Node features after GCN layer:
tensor([[0.0678, 0.0000],
        [0.1420, 0.0000],
        [0.2764, 0.0000]], grad_fn=<ReluBackward0>)


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import numpy as np

# Example: Simulated Mobility Data for FANET nodes
num_nodes = 5
positions = np.array([[0, 0], [5, 5], [10, 10], [15, 15], [20, 20]])  # [x, y] positions
velocities = np.array([3, 2, 1, 4, 3])  # Node velocities
link_quality = np.array([0.9, 0.8, 0.7, 0.85, 0.9])  # Link quality between nodes
queue_sizes = np.array([5, 10, 8, 15, 12])  # Queue sizes

# Step 2: Feature Set for GCN (combine mobility features)
features = np.column_stack((positions, velocities, link_quality, queue_sizes))  # Node features
features = torch.tensor(features, dtype=torch.float)

# Step 3: Define Graph Structure (Adjacency matrix)
# Here we simulate a simple topology where nodes are connected to their neighbors
edge_index = torch.tensor([[0, 1, 2, 3, 4],  # From node 1, 2, 3, 4, 5
                           [1, 0, 1, 2, 3]], dtype=torch.long)  # To node 2, 1, 3, 4, 5

# Step 4: Define GCN Model for Routing Prediction
class GCNRouting(nn.Module):
    def __init__(self):
        super(GCNRouting, self).__init__()
        # Input features should match the number of features in your data
        self.conv1 = GCNConv(5, 16)  # Changed input features to 5
        # Output next hop prediction (num_nodes classes)
        self.conv2 = GCNConv(16, num_nodes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x
# Step 5: Create the graph data
data = Data(x=features, edge_index=edge_index)

# Step 6: Train the GCN for Routing Prediction
model = GCNRouting()

# Simulated next hop labels for training (assumed for this example)
# Let's assume node 0 should route through node 1, node 1 through node 2, etc.
next_hops = torch.tensor([1, 2, 3, 4, 0])  # Next hop node for each node

# Step 7: Define Loss and Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

# Step 8: Training Loop (Simplified for illustration)
for epoch in range(100):
    optimizer.zero_grad()
    out = model(data)
    loss = loss_fn(out, next_hops)  # Compare predicted next hop with true next hop
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f'Epoch {epoch}: Loss = {loss.item():.4f}')

# Step 9: Prediction after training
model.eval()
output = model(data)
predicted_next_hops = output.argmax(dim=1)  # Predicted next hop for each node
print(f"Predicted next hops: {predicted_next_hops}")


Epoch 0: Loss = 6.1359
Epoch 10: Loss = 1.5046
Epoch 20: Loss = 0.8959
Epoch 30: Loss = 0.7767
Epoch 40: Loss = 0.6376
Epoch 50: Loss = 0.5548
Epoch 60: Loss = 0.4804
Epoch 70: Loss = 0.4204
Epoch 80: Loss = 0.3687
Epoch 90: Loss = 0.3252
Predicted next hops: tensor([1, 2, 3, 4, 0])


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import numpy as np

# Example: Simulated Mobility Data for FANET nodes
num_nodes = 5
positions = np.array([[0, 0], [5, 5], [10, 10], [15, 15], [20, 20]])  # [x, y] positions
velocities = np.array([3, 2, 1, 4, 3])  # Node velocities
link_quality = np.array([0.9, 0.8, 0.7, 0.85, 0.9])  # Link quality between nodes
queue_sizes = np.array([5, 10, 8, 15, 12])  # Queue sizes

# Step 2: Feature Set for GCN (combine mobility features)
features = np.column_stack((positions, velocities, link_quality, queue_sizes))  # Node features
features = torch.tensor(features, dtype=torch.float)

# Step 3: Define Graph Structure (Adjacency matrix)
# Simulated topology with multi-hop edges (e.g., node 0 connected to nodes 1, 2, and 3)
edge_index = torch.tensor([[0, 1, 2, 3, 4, 0, 2, 3, 4],
                           [1, 0, 0, 1, 2, 2, 3, 4, 4]], dtype=torch.long)

# Step 4: Define Multi-layer GCN Model for Routing Prediction
class MultiLayerGCN(nn.Module):
    def __init__(self, num_layers, input_dim, hidden_dim, output_dim):
        super(MultiLayerGCN, self).__init__()
        self.num_layers = num_layers
        self.convs = nn.ModuleList()
        self.convs.append(GCNConv(input_dim, hidden_dim))

        # Add additional layers
        for _ in range(num_layers - 1):
            self.convs.append(GCNConv(hidden_dim, hidden_dim))

        self.fc = nn.Linear(hidden_dim, output_dim)  # Output layer for next hop prediction

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        for conv in self.convs:
            x = conv(x, edge_index)
            x = F.relu(x)
        x = self.fc(x)  # Output next-hop prediction
        return x

# Step 5: Create the graph data
data = Data(x=features, edge_index=edge_index)

# Step 6: Initialize Model with 3 Layers (you can adjust the number of layers)
# Change input_dim to 5 to match the number of features, and output_dim to num_nodes
model = MultiLayerGCN(num_layers=3, input_dim=5, hidden_dim=16, output_dim=num_nodes)

# Step 7: Simulated next hop labels for training (assumed for this example)
next_hops = torch.tensor([1, 2, 3, 4, 0])  # Next hop node for each node

# Step 8: Define Loss and Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

# Step 9: Training Loop (Simplified for illustration)
for epoch in range(100):
    optimizer.zero_grad()
    out = model(data)
    loss = loss_fn(out, next_hops)  # Compare predicted next hop with true next hop
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f'Epoch {epoch}: Loss = {loss.item():.4f}')

# Step 10: Prediction after training
model.eval()
output = model(data)
predicted_next_hops = output.argmax(dim=1)  # Predicted next hop for each node
print(f"Predicted next hops: {predicted_next_hops}")


Epoch 0: Loss = 2.3203
Epoch 10: Loss = 1.6169
Epoch 20: Loss = 1.5850
Epoch 30: Loss = 1.4991
Epoch 40: Loss = 1.1685
Epoch 50: Loss = 0.8827
Epoch 60: Loss = 0.7081
Epoch 70: Loss = 0.5786
Epoch 80: Loss = 0.4894
Epoch 90: Loss = 0.4444
Predicted next hops: tensor([1, 1, 3, 4, 0])
